## Playground

### Todo
- [x] eine Funktion die ein Lat/Long tuple annimmt und die Urls der n nächsten Radiostationen zurückgibt.

- [ ] eine Funktion die die Koordinaten aus dem erhaltenen Format zu Lat/Long übersetzt.

- [x] eine Funktion die die ID einer gewählten Radiostation annimmt und den Audiostream dieser Station ausgibt.

- [ ] eine Funktion die den Videostream einer Webcam annimmt und Koordinaten ausliest (auf welche Art auch immer).

In [ ]:
import json
import requests

'''
getting basic list of places. geo field ist a tuple of shape [Long, Lat]
'''

r = requests.get('http://radio.garden/api/ara/content/places', headers={'Accept': 'application/json'})
r_json = r.json()
place_list = r_json["data"]["list"]

In [ ]:
import geopy.distance

'''
distances of places to given lat/long tuple
'''

pin_location = (48.2083537,16.3725042) #Vienna
n_stations = 247
distance_list = [dict(place, distance=geopy.distance.geodesic((place["geo"][1],place["geo"][0]), pin_location).km)  for place in place_list]

In [ ]:
'''
Sorting by closest places to given lat/long tuple & selecting the until n amount of radio stations is reached
'''

distance_list = sorted(distance_list, key=lambda x: x['distance'])
selected_places = []
for place in distance_list:
    if sum(selected_place['size'] for selected_place in selected_places) < n_stations:
        selected_places.append(place)

In [ ]:
import random
from datetime import datetime

'''
selecting n radio stations from the selected places
'''

random.seed(str(datetime.now())) #random seed generated by system time
selected_channels = []
for place in selected_places:
    #radio stations are not saved within the place data therefore a new request must be sent per selected place
    r = requests.get(f'http://radio.garden/api/ara/content/page/{place["id"]}/channels', headers={'Accept': 'application/json'}) 
    r_json = r.json()

    #if the furthest of the selected places contains more stations than requested by n, we randomly select until n is met
    if place == selected_places[len(selected_places)-1] and place["size"] > 1:
        for channel in random.sample(r_json["data"]["content"][0]["items"], n_stations-len(selected_channels)):
            selected_channels.append(channel)

    else:
        for channel in r_json["data"]["content"][0]["items"]:
            selected_channels.append(channel)

In [ ]:
import vlc

'''
getting audio output from Station ID
'''

some_ID = "3WpG40FL" #OE1
  
url =f"http://radio.garden/api/ara/content/listen/{some_ID}/channel.mp3"


instance = vlc.Instance('--input-repeat=-1', '--fullscreen')

#Define VLC player
player=instance.media_player_new()

#Define VLC media
media=instance.media_new(url)

#Set player media
player.set_media(media)

#Play the media
player.play()

#Pause the media

#player.pause()

#Stop the media

#player.stop()

In [43]:
%matplotlib tk
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

mpl.rcParams['toolbar'] = 'None'

fig = plt.figure()


img=mpimg.imread('plattkarte_stretched.jpg')

def onclick(event):
    ix, iy = event.xdata, event.ydata
    print(ix, iy)

cid = fig.canvas.mpl_connect('button_press_event', onclick)

imgplot = plt.imshow(img)
imgplot.axes.get_xaxis().set_visible(False)
imgplot.axes.get_yaxis().set_visible(False)


mng = plt.get_current_fig_manager()
### works on Ubuntu??? >> did NOT working on windows
# mng.resize(*mng.window.maxsize())
 #works fine on Windows!
mng.full_screen_toggle()
#fig.tight_layout(pad=0) #this is wonky af. 
plt.subplots_adjust(left=0, right=1, top=1, bottom=0, wspace=0) #this has to be set individually to screen

plt.show()